In [1]:
import glob
import os

eval_folder = "/homes/kb658/fusion/evaluations/"

all_json_files = glob.glob(os.path.join(eval_folder, "**/*.json"), recursive=True)

## Comparison with AMT

In [ ]:
experiment_5_files = [f for f in all_json_files if "experiment_5" in f]
experiment_5_directories = set([os.path.dirname(f) for f in experiment_5_files])

experiment_6_files = [f for f in all_json_files if "experiment_6" in f]
experiment_6_directories = set([os.path.dirname(f) for f in experiment_5_files])

In [ ]:
import json
import math
import pprint

def get_metrics_from_file(filepaths):
    results = {'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'pitch_class_kl': [], 
            'pctm_cosine_sim': [], 
            'avg_ioi': []}

    for f in filepaths:
        with open(f, "r") as f:
            metrics = json.load(f)

        for k in results.keys():
            results[k].append(metrics[k])

    # Average num notes, note density, used pitch, ioi element wise in list
    avg_num_notes_original = sum([i[0] for i in results['num_notes']]) / len(results['num_notes'])
    avg_num_notes_generated = sum([i[1] for i in results['num_notes']]) / len(results['num_notes'])

    avg_note_density_original = sum([i[0] for i in results['note_density']]) / len(results['note_density'])
    avg_note_density_generated = sum([i[1] for i in results['note_density']]) / len(results['note_density'])

    avg_used_pitch_original = sum([i[0] for i in results['used_pitch']]) / len(results['used_pitch'])
    avg_used_pitch_generated = sum([i[1] for i in results['used_pitch']]) / len(results['used_pitch'])

    avg_ioi_original = sum([i[0] for i in results['avg_ioi']]) / len(results['avg_ioi'])
    avg_ioi_generated = sum([i[1] for i in results['avg_ioi']]) / len(results['avg_ioi'])

    avg_pitch_class_kl = sum([i for i in results['pitch_class_kl']]) / len(results['pitch_class_kl'])

    # Filter out NaN values
    results['pctm_cosine_sim'] = [i for i in results['pctm_cosine_sim'] if not math.isnan(i)]
    avg_pctm_cosine_sim = sum([i for i in results['pctm_cosine_sim']]) / len(results['pctm_cosine_sim'])

    # Add averages to results
    results = {
        'num_notes': (avg_num_notes_original, avg_num_notes_generated),
        'note_density': (avg_note_density_original, avg_note_density_generated),
        'used_pitch': (avg_used_pitch_original, avg_used_pitch_generated),
        'pitch_class_kl': avg_pitch_class_kl,
        'pctm_cosine_sim': avg_pctm_cosine_sim,
        'avg_ioi': (avg_ioi_original, avg_ioi_generated)
    }

    return results

In [19]:
amt_filepaths = [os.path.join(f, "amt_metrics.json") for f in experiment_5_directories]
improvnet_filepaths = [os.path.join(f, "metrics.json") for f in experiment_5_directories]

amt_metrics = get_metrics_from_file(amt_filepaths)
improvnet_metrics = get_metrics_from_file(improvnet_filepaths)

pprint.pprint(amt_metrics)
pprint.pprint(improvnet_metrics)

{'avg_ioi': (0.13942976033627477, 0.1507879825995296),
 'note_density': (32.43775100401606, 49.626506024096386),
 'num_notes': (87.81927710843374, 99.25301204819277),
 'pctm_cosine_sim': 0.36239926818593476,
 'pitch_class_kl': 1.4593494362096295,
 'used_pitch': (27.879518072289155, 25.44578313253012)}
{'avg_ioi': (0.13942976033627477, 0.1224134601615607),
 'note_density': (32.43775100401606, 36.30120481927712),
 'num_notes': (87.81927710843374, 95.62650602409639),
 'pctm_cosine_sim': 0.4036273144209619,
 'pitch_class_kl': 0.8906669396230863,
 'used_pitch': (27.879518072289155, 28.783132530120483)}


In [22]:
amt_filepaths = [os.path.join(f, "amt_metrics.json") for f in experiment_6_directories]
improvnet_filepaths = [os.path.join(f, "metrics.json") for f in experiment_6_directories]

amt_metrics = get_metrics_from_file(amt_filepaths)
improvnet_metrics = get_metrics_from_file(improvnet_filepaths)

pprint.pprint(amt_metrics)
pprint.pprint(improvnet_metrics)

{'avg_ioi': (0.13942976033627477, 0.1507879825995296),
 'note_density': (32.43775100401606, 49.626506024096386),
 'num_notes': (87.81927710843374, 99.25301204819277),
 'pctm_cosine_sim': 0.36239926818593476,
 'pitch_class_kl': 1.4593494362096295,
 'used_pitch': (27.879518072289155, 25.44578313253012)}
{'avg_ioi': (0.13942976033627477, 0.1224134601615607),
 'note_density': (32.43775100401606, 36.30120481927712),
 'num_notes': (87.81927710843374, 95.62650602409639),
 'pctm_cosine_sim': 0.4036273144209619,
 'pitch_class_kl': 0.8906669396230863,
 'used_pitch': (27.879518072289155, 28.783132530120483)}


## Harmony

In [44]:
import json
import pprint

def get_harmony_metrics(harmony_files):

    generated_results = {
            'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'pitch_class_kl': [], 
            'pctm_cosine_sim': [], 
            'avg_ioi': [],
            'n_pitches': [],
            'n_unique_chords': [],
            'pitch_in_scale_rate': [],
            'pitch_range': [],
            'polyphony': [],
            'polyphony_rate': [],
            'groove_consistency': [],
            'tonal_tension_diameter': [],
            'tonal_tension_tensile': [],
            'tonal_tension_centroid': []
            }

    original_results = {
            'num_notes': [], 
            'note_density': [], 
            'used_pitch': [], 
            'avg_ioi': [],
            'n_pitches': [],
            'n_unique_chords': [],
            'pitch_in_scale_rate': [],
            'pitch_range': [],
            'polyphony': [],
            'polyphony_rate': [],
            'groove_consistency': [],
            'tonal_tension_diameter': [],
            'tonal_tension_tensile': [],
            'tonal_tension_centroid': []
            }
    
    monophonic_results = {
            'num_notes': [], 
            'note_density': []
            }
    
    for file in harmony_files:
        with open(file, "r") as f:
            metrics = json.load(f)

        for k in generated_results.keys():
            if k in metrics['Metric Pairs']:
                if type(metrics['Metric Pairs'][k]) == list:
                    generated_results[k].append(metrics['Metric Pairs'][k][1])
                else:
                    generated_results[k].append(metrics['Metric Pairs'][k])
            elif k in metrics['Generated MusPy Metrics']["pitch_related"]:
                generated_results[k].append(metrics['Generated MusPy Metrics']["pitch_related"][k])
            elif k in metrics['Generated MusPy Metrics']["rhythm_related"]:
                generated_results[k].append(metrics['Generated MusPy Metrics']["rhythm_related"][k])
            elif "centroid" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Diameter"])
            elif "diameter" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Tensile"])
            elif "tensile" in k:
                generated_results[k].append(metrics["Generated Tonal Tension Centroid Diff"])

        for k in original_results.keys():
            if k in metrics['Metric Pairs']:
                if type(metrics['Metric Pairs'][k]) == list:
                    original_results[k].append(metrics['Metric Pairs'][k][0])
            elif k in metrics['Original MusPy Metrics']["pitch_related"]:
                original_results[k].append(metrics['Original MusPy Metrics']["pitch_related"][k])
            elif k in metrics['Original MusPy Metrics']["rhythm_related"]:
                original_results[k].append(metrics['Original MusPy Metrics']["rhythm_related"][k])
            elif "centroid" in k:
                original_results[k].append(metrics["Original Tonal Tension Diameter"])
            elif "diameter" in k:
                original_results[k].append(metrics["Original Tonal Tension Tensile"])
            elif "tensile" in k:
                original_results[k].append(metrics["Original Tonal Tension Centroid Diff"])

        for k in monophonic_results.keys():
            monophonic_results[k].append(metrics['Monophonic Metric Pairs'][k][0])

    # Average all values in list
    for k in generated_results.keys():
        generated_results[k] = sum(generated_results[k]) / len(generated_results[k])
    for k in original_results.keys():
        original_results[k] = sum(original_results[k]) / len(original_results[k])
    for k in monophonic_results.keys():
        monophonic_results[k] = sum(monophonic_results[k]) / len(monophonic_results[k])
                
    return generated_results, original_results, monophonic_results

In [41]:
harmony_files = [f for f in all_json_files if "harmony" in f]
random_chords_files_constraints = [f for f in harmony_files if "random_chords" in f and "constraints" in f]
generated_files_constraints = [f for f in harmony_files if "random_chords" not in f and "with_constraints" in f]
generated_files_no_constraints = [f for f in harmony_files if "random_chords" not in f and "without_constraints" in f]

# Print length of all files
print(len(harmony_files))
print(len(random_chords_files_constraints))
print(len(generated_files_constraints))
print(len(generated_files_no_constraints))

3500
1165
1165
1170


In [45]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.15958372158366999,
 'groove_consistency': 0.9918497280466027,
 'n_pitches': 36.89613733905579,
 'n_unique_chords': 19.783690987124462,
 'note_density': 35.21692983789978,
 'num_notes': 571.1055793991417,
 'pctm_cosine_sim': 0.7773760505969426,
 'pitch_class_kl': 0.34299783399993283,
 'pitch_in_scale_rate': 0.7909207769866156,
 'pitch_range': 51.33991416309013,
 'polyphony': 10.740271742056516,
 'polyphony_rate': 0.9172381243128878,
 'tonal_tension_centroid': 1.9883007406708597,
 'tonal_tension_diameter': 0.6251597516814147,
 'tonal_tension_tensile': 0.5338005474457509,
 'used_pitch': 36.89613733905579}
{'avg_ioi': 0.2876748630915622,
 'groove_consistency': 0.9998113478076276,
 'n_pitches': 25.84549356223176,
 'n_unique_chords': 13.278969957081545,
 'note_density': 19.154030032820796,
 'num_notes': 303.3948497854077,
 'pitch_in_scale_rate': 0.7945453516812427,
 'pitch_range': 34.68669527896996,
 'polyphony': 13.317558101106977,
 'polyphony_rate': 0.957771440843409,
 'tonal

In [46]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(generated_files_no_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.47732037329333765,
 'groove_consistency': 0.9959658423131761,
 'n_pitches': 18.32905982905983,
 'n_unique_chords': 14.011965811965812,
 'note_density': 11.187630341648058,
 'num_notes': 170.22735042735042,
 'pctm_cosine_sim': 0.7229398884222135,
 'pitch_class_kl': 0.5455414873133614,
 'pitch_in_scale_rate': 0.8016767966622677,
 'pitch_range': 29.41025641025641,
 'polyphony': 2.440297748230915,
 'polyphony_rate': 0.23969971361490583,
 'tonal_tension_centroid': 0.4561096059882356,
 'tonal_tension_diameter': 0.7556659796708923,
 'tonal_tension_tensile': 0.7753827234513683,
 'used_pitch': 18.32905982905983}
{'avg_ioi': 0.2888099221746094,
 'groove_consistency': 0.9998117128673034,
 'n_pitches': 25.82905982905983,
 'n_unique_chords': 13.264957264957266,
 'note_density': 19.10721793866345,
 'num_notes': 302.62393162393164,
 'pitch_in_scale_rate': 0.7950462890923584,
 'pitch_range': 34.68376068376068,
 'polyphony': 13.306769833359834,
 'polyphony_rate': 0.9578652796848034,
 'ton

In [47]:
generated_metrics, original_metrics, monophonic_results = get_harmony_metrics(random_chords_files_constraints)
pprint.pprint(generated_metrics)
pprint.pprint(original_metrics)
pprint.pprint(monophonic_results)

{'avg_ioi': 0.15958372158366999,
 'groove_consistency': 0.9918492817080373,
 'n_pitches': 55.50128755364807,
 'n_unique_chords': 37.53304721030043,
 'note_density': 35.22023251519387,
 'num_notes': 571.1055793991417,
 'pctm_cosine_sim': 0.628860801019517,
 'pitch_class_kl': 0.3200465146111745,
 'pitch_in_scale_rate': 0.6379698697728563,
 'pitch_range': 63.75278969957082,
 'polyphony': 5.266617109250974,
 'polyphony_rate': 0.7536260033875086,
 'tonal_tension_centroid': 2.860866738298016,
 'tonal_tension_diameter': 0.9139033641575995,
 'tonal_tension_tensile': 0.5222243531495426,
 'used_pitch': 55.50128755364807}
{'avg_ioi': 0.2876748630915622,
 'groove_consistency': 0.9998113478076276,
 'n_pitches': 25.84549356223176,
 'n_unique_chords': 13.278969957081545,
 'note_density': 19.154030032820796,
 'num_notes': 303.3948497854077,
 'pitch_in_scale_rate': 0.7945453516812427,
 'pitch_range': 34.68669527896996,
 'polyphony': 13.317558101106977,
 'polyphony_rate': 0.957771440843409,
 'tonal_tens